In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample-cap/new_clip_embeddings_part_0.pkl
/kaggle/input/sample-cap/sample.csv
/kaggle/input/sample/000000000009.jpg
/kaggle/input/sample/000000000025.jpg


In [9]:
!pip -q install -U bitsandbytes peft

In [10]:
!pip install --upgrade huggingface_hub

In [11]:
import pickle

# Specify the path to your .pkl file
file_path = '/kaggle/input/sub-capstone/sub_clip_embeddings_0.pkl'

# Load the embeddings from the .pkl file
with open(file_path, 'rb') as file:
    embeddings = pickle.load(file)


# import pickle

# # Specify the path to your .pkl file
# file_path = '/kaggle/input/llava-processed/final_clip_embeddings_part_1.pkl'

# # Load the embeddings from the .pkl file
# with open(file_path, 'rb') as file:
#     embeddings_1 = pickle.load(file)


In [13]:
len(embeddings)

50

In [7]:
from sys import getsizeof


getsizeof(embeddings)

In [16]:
select_feature = 'patch'
def feature_select(image_forward_outs):
    image_features = image_forward_outs.hidden_states[-1]
    if select_feature == 'patch':
        image_features = image_features[:, 1:]  # Skip CLS token if selecting patch
    elif select_feature == 'cls_patch':
        image_features = image_features  # Keep CLS + patch tokens
    else:
        raise ValueError(f'Unexpected select feature: {select_feature}')
    return image_features

In [17]:
from huggingface_hub import login
login()

In [18]:
# meta-llama/Llama-3.2-1B-Instruct
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch
# Load PHI 2 model with 4-bit quantization for efficient fine-tuning
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )
model_name = "meta-llama/Llama-3.2-1B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name, 
#     quantization_config=bnb_config,
    trust_remote_code=True
)


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [181]:
import torch.nn as nn
import random
class MLPProjection(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=1024, depth=2):
        super(MLPProjection, self).__init__()
        modules = []
        modules.append(nn.Linear(input_dim, hidden_dim,bias = False))
        
        for _ in range(1, depth):
            modules.append(nn.GELU())
            modules.append(nn.Linear(hidden_dim, output_dim,bias=False))
        
        self.mlp = nn.Sequential(*modules)
    
    def forward(self, x):
        return self.mlp(x)

class PHI2WithMLP(nn.Module):
    def __init__(self, phi2_model, mlp_projection):
        super(PHI2WithMLP, self).__init__()
        self.phi2_model = phi2_model
        self.mlp_projection = mlp_projection
        self.config = phi2_model.config

    def forward(self, image_embeddings=None,
                inputs_embeds=None,
                input_ids=None,
                attention_mask=None,
                labels=None,
                output_attentions=False, 
        output_hidden_states=False, 
        **kwargs):  # Catch any additional arguments):
        
        if input_ids is not None:
            token_embeddings = self.phi2_model.get_input_embeddings()(input_ids)
        elif inputs_embeds is not None:
            token_embeddings = inputs_embeds
        else:
            raise ValueError("You must provide either input_ids or inputs_embeds.")

        
        if image_embeddings is not None:
            # Apply MLP to image embeddings to map to text embedding space
            projected_image_embeddings = self.mlp_projection(image_embeddings)

            # Get the sequence length for the image embeddings
            image_embedding_length = projected_image_embeddings.size(1)
            
            batch_size, text_sequence_length = attention_mask.shape
            
            # Extend attention mask for image embeddings (ones for image embedding positions)
            new_attention_mask = torch.cat(
                [attention_mask, torch.ones((batch_size,image_embedding_length), device=attention_mask.device)], dim=1
            )
            
            # Combine image and token embeddings
            combined_embeddings = torch.cat([projected_image_embeddings, token_embeddings], dim=1)  # Concatenating along sequence length
            
        else:
            # No image embeddings: Use only token embeddings and the original attention mask
            combined_embeddings = token_embeddings
            new_attention_mask = attention_mask
        if labels is not None:
            # Labels should match the sequence length of combined embeddings
            # If labels correspond only to text tokens, pad them to match the new sequence length
            if image_embeddings is not None:
                label_padding = torch.full(
                    (batch_size, image_embedding_length), -100, device=labels.device  # Use -100 for ignore index
                )
                new_labels = torch.cat([label_padding,labels], dim=1)
            else:
                new_labels = labels
        else:
            new_labels = labels
        # Pass the combined embeddings through the PHI2 model with the (updated or original) attention mask
        outputs = self.phi2_model(inputs_embeds=combined_embeddings, attention_mask=new_attention_mask,labels = new_labels, output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            **kwargs)

        return outputs
    
    def prepare_inputs_for_generation(self, input_ids, attention_mask=None, image_embeddings=None, **kwargs):
        # Generate inputs with projections where necessary
        if image_embeddings is not None:
            projected_image_embeddings = self.mlp_projection(image_embeddings)
            token_embeddings = self.phi2_model.get_input_embeddings()(input_ids)
            combined_embeddings = torch.cat([projected_image_embeddings, token_embeddings], dim=1)
            new_attention_mask = torch.cat([torch.ones_like(projected_image_embeddings[..., :1]), attention_mask], dim=1)
        else:
            combined_embeddings = self.phi2_model.get_input_embeddings()(input_ids)
            new_attention_mask = attention_mask

        return {
            "inputs_embeds": combined_embeddings,
            "attention_mask": new_attention_mask,
            **kwargs
        }

def create_phi2_model_with_lora(mlp_projection,lan_model):
    
    for param in mlp_projection.parameters():
        param.requires_grad = True

    # Return PHI2 model with MLP projection
    return PHI2WithMLP(lan_model, mlp_projection)
    
model_embedding_dim = model.config.hidden_size  # This might change based on your model architecture

# Example usage
input_dim = 768  # Input dimension of image embeddings
output_dim = model_embedding_dim  # Target dimension of text embeddings
hidden_dim = 1024  # Hidden layer dimension of the MLP

mlp_projection = MLPProjection(input_dim, output_dim, hidden_dim, depth=2)  # Customize MLP
combined_model = create_phi2_model_with_lora(mlp_projection, model)


from peft import LoraModel, LoraConfig,get_peft_model

# Set up the QLoRA configuration for attention layers in PHI 2
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Apply QLoRA only to these layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap PHI 2 with QLoRA
# phi_lora_model = LoraModel(model, lora_config,"default")




phi_lora_model = get_peft_model(combined_model, lora_config)

In [182]:
phi_lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PHI2WithMLP(
      (phi2_model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 2048)
          (layers): ModuleList(
            (0-15): 16 x LlamaDecoderLayer(
              (self_attn): LlamaSdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=2048, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_

In [183]:
phi_lora_model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 1,240,401,920 || trainable%: 0.1374


In [184]:
for name, param in phi_lora_model.named_parameters():
    if 'mlp_projection' in name :
        param.requires_grad = True

In [185]:
phi_lora_model.print_trainable_parameters()

trainable params: 4,587,520 || all params: 1,240,401,920 || trainable%: 0.3698


In [186]:
from transformers import CLIPModel

# Load CLIP and PHI2
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

In [187]:

# # Now the model can be trained, and the optimizer only updates LoRA and projection
# optimizer = torch.optim.AdamW(
#     [p for p in combined_model.parameters() if p.requires_grad], lr=1e-4
# )

In [188]:

# # Training loop
# def train_model(combined_model, data_loader, optimizer, num_epochs=1, device="cuda"):
#     combined_model.train()
#     combined_model = combined_model.to(device)
    
#     for epoch in range(num_epochs):
#         total_loss = 0
#         for batch in data_loader:
#             image_embeddings = batch['image_embeddings'].to(device)
#             input_ids = batch['input_ids'].to(device)
#             labels = batch['labels'].to(device)
            
#             # Forward pass
#             optimizer.zero_grad()
#             outputs = combined_model(image_embeddings, input_ids)
            
#             # Assume outputs is a tuple where the first element is logits
#             logits = outputs.logits
            
#             # Flatten the logits and labels for cross-entropy loss
#             logits = logits.view(-1, logits.size(-1))
#             labels = labels.view(-1)
            
#             # Calculate loss (cross-entropy loss for language modeling)
#             loss = F.cross_entropy(logits, labels)
#             total_loss += loss.item()
            
#             # Backward pass and optimization
#             loss.backward()
#             optimizer.step()
        
#         avg_loss = total_loss / len(data_loader)
#         print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

# # Usage
# data_loader = get_data_loader(batch_size=16)  # Adjust the batch size as needed
# train_model(combined_model, data_loader, optimizer, num_epochs=1)


In [189]:
# for batch in data_loader:
#     image_embeddings = batch['image_embeddings'].to(device)  # Assuming pre-extracted embeddings
#     input_ids = batch['input_ids'].to(device)  # Tokenized text input
#     labels = batch['labels'].to(device)  # Labels for training
    
#     # Forward pass through the model
#     optimizer.zero_grad()
#     outputs = combined_model(image_embeddings, input_ids)
    
#     # Get logits and calculate loss
#     logits = outputs.logits.view(-1, logits.size(-1))
#     labels = labels.view(-1)
#     loss = F.cross_entropy(logits, labels)
    
#     # Backward pass and optimization
#     loss.backward()
#     optimizer.step()


In [190]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [191]:
# import pickle

# # Specify the path to your .pkl file
# file_path = '/kaggle/input/sample-cap/new_clip_embeddings_part_0.pkl'

# # Load the embeddings from the .pkl file
# with open(file_path, 'rb') as file:
#     embeddings = pickle.load(file)

# # Now you can use your embeddings
# # print(embeddings)


In [192]:
df = pd.read_csv('/kaggle/input/sample-cap/turns_50_sample.csv')

In [193]:
# import torch
# from transformers import CLIPProcessor, CLIPModel
# from transformers import Trainer, TrainingArguments
# from datasets import Dataset
# from torch.utils.data import Dataset as TorchDataset
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from PIL import Image

# # Initialize the tokenizer and image model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token
# clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
# clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

# class CustomDataset(TorchDataset):
#     def __init__(self, image_paths, text_inputs, text_labels):
#         self.image_paths = image_paths
#         self.text_inputs = text_inputs
#         self.text_labels = text_labels
#         self.max_length = 256

#     def __len__(self):
#         return len(self.text_labels)

#     def __getitem__(self, idx):
#         image = Image.open(self.image_paths[idx])
#         inputs = clip_processor(images=image, return_tensors="pt")
#         image_forward_outs = clip_model.vision_model(**inputs, output_hidden_states=True)
#         image_features = feature_select(image_forward_outs)
#         image_embedding = image_features.squeeze(0).to(device)
        
#         # Tokenize text input
#         input_encoding = tokenizer(
#             self.text_inputs[idx].replace('<image>',' '),
#             return_tensors='pt',
#             padding='max_length',  # Pad to max length
#             truncation=True,  # Truncate if needed
#             max_length=self.max_length
#         )
        
#         # Tokenize text label (similar to inputs)
#         label_encoding = tokenizer(
#             self.text_labels[idx],
#             return_tensors='pt',
#             padding='max_length',
#             truncation=True,
#             max_length=self.max_length
#         )

#         # Extract input_ids and attention_mask for both inputs and labels
#         input_ids = input_encoding['input_ids'].squeeze(0)
#         input_attention_mask = input_encoding['attention_mask'].squeeze(0)
#         label_ids = label_encoding['input_ids'].squeeze(0)
#         label_attention_mask = label_encoding['attention_mask'].squeeze(0)
        
        
#         # Return the image embeddings, tokenized inputs/labels, and attention masks
#         return {
#             'image_embeddings': image_embedding,  # Precomputed image embedding
#             'input_ids': input_ids,  # Tokenized input
#             'attention_mask': input_attention_mask,  # Attention mask for input
#             'labels': label_ids,  # Tokenized label
# #             'label_attention_mask': label_attention_mask  # Attention mask for label (optional)
#         }

# # Create dataset (you will replace this with actual paths and data)
# image_paths = ["/kaggle/input/sample/000000000009.jpg", "/kaggle/input/sample/000000000009.jpg"]
# text_inputs = ["What is the capital of France?", "Describe a sunset."]
# text_labels = ["Paris", "A beautiful view at dusk."]  # Example text labels

# # Instantiate dataset
# dataset = CustomDataset(image_paths, text_inputs, text_labels)



In [198]:
import torch
from transformers import CLIPProcessor, CLIPModel
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from torch.utils.data import Dataset as TorchDataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image

# Initialize the tokenizer and image model
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
clip_model = CLIPModel.from_pretrained('openai/clip-vit-base-patch32')
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch32')

class CustomDataset(TorchDataset):
    def __init__(self, image_paths, text_inputs, text_labels):
        self.image_paths = image_paths
        self.text_inputs = text_inputs
        self.text_labels = text_labels
        self.max_length = 256

    def __len__(self):
        return len(self.text_labels)

    def __getitem__(self, idx):
        image_embedding = embeddings[self.image_paths[idx]]
        
        # Tokenize text input
        input_encoding = tokenizer(
            self.text_inputs[idx].replace('<image>','')+self.text_labels[idx],
            return_tensors='pt',
            padding='max_length',  # Pad to max length
            truncation=True,  # Truncate if needed
            max_length=self.max_length
        )
        
        # Tokenize text label (similar to inputs)
        label_encoding = tokenizer(
            self.text_inputs[idx].replace('<image>','')+self.text_labels[idx],
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.max_length
        )

        # Extract input_ids and attention_mask for both inputs and labels
        input_ids = input_encoding['input_ids'].squeeze(0)
        input_attention_mask = input_encoding['attention_mask'].squeeze(0)
        label_ids = label_encoding['input_ids'].squeeze(0)
        label_attention_mask = label_encoding['attention_mask'].squeeze(0)
        
        
        # Return the image embeddings, tokenized inputs/labels, and attention masks
        return {
            'image_embeddings': image_embedding,  # Precomputed image embedding
            'input_ids': input_ids,  # Tokenized input
            'attention_mask': input_attention_mask,  # Attention mask for input
            'labels': label_ids,  # Tokenized label
#             'label_attention_mask': label_attention_mask  # Attention mask for label (optional)
        }

# Create dataset (you will replace this with actual paths and data)
image_paths = df['image'].tolist()
text_inputs = df['input'].tolist()
text_labels = df['label'].tolist()  # Example text labels

# Instantiate dataset
dataset = CustomDataset(image_paths, text_inputs, text_labels)

In [199]:

import wandb
wandb.init(mode="disabled")

In [200]:
from transformers import DataCollatorWithPadding

class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.default_collator = DataCollatorWithPadding(tokenizer)

    def __call__(self, features):
        # Extract input features (image embeddings, text inputs, etc.)
        input_ids = [f['input_ids'] for f in features]
        attention_mask = [f['attention_mask'] for f in features]
        image_embeddings = [f['image_embeddings'] for f in features if 'image_embeddings' in f]
        labels = [f['labels'] for f in features if 'labels' in f]

        # Collate the text inputs using the default collator
        batch = self.default_collator(features)

        # Add image embeddings if they exist
        if image_embeddings:
            batch['image_embeddings'] = torch.stack(image_embeddings)

        # Add labels to the batch
        if labels:
            batch['labels'] = torch.stack(labels)

        return batch


In [201]:

from transformers import TrainingArguments



training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    remove_unused_columns=False,
)
# mlp_projection = MLPProjection(input_dim=512, output_dim=768, hidden_dim=1024, num_layers=2)
# model = PHI2WithMLP(mlp_projection,phi_lora_model = phi_lora_model)
from transformers import DataCollatorWithPadding

# Create a data collator
data_collator = CustomDataCollator(tokenizer=tokenizer)

# Create Trainer
trainer = Trainer(
    model=phi_lora_model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,  # Use the collator
)

# Start training
trainer.train()


Step,Training Loss


TrainOutput(global_step=57, training_loss=1.3543715895267956, metrics={'train_runtime': 2761.5686, 'train_samples_per_second': 0.081, 'train_steps_per_second': 0.021, 'total_flos': 0.0, 'train_loss': 1.3543715895267956, 'epoch': 1.0})

In [202]:
image_paths = df['image'].tolist()
text_inputs = df['input'].tolist()
text_labels = df['label'].tolist()

In [203]:
dataset[0]

{'image_embeddings': tensor([[ 0.3817,  0.9930,  0.2789,  ...,  0.0197, -0.0689, -0.4641],
         [ 0.3424,  0.9193,  1.4041,  ..., -0.1432, -0.2891,  0.5565],
         [-0.3738,  0.6973,  0.2574,  ...,  0.1940,  0.2922,  0.6109],
         ...,
         [ 0.3682,  0.4731, -0.4251,  ...,  0.5928,  0.2523, -0.6850],
         [-0.1267,  0.3708, -0.3366,  ...,  0.1251,  0.5551, -0.3766],
         [ 0.5355,  1.0992, -0.4195,  ...,  0.3650, -0.2440,  0.3265]]),
 'input_ids': tensor([128000,    198,   3923,    527,    279,   8146,    315,    279,   5951,
            304,    279,   2217,     30,    791,   5951,    304,    279,   2217,
            374,   4251,    323,   2579,     13, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009

In [204]:
image_path,text_input,text_label # Example text label (if needed for comparison)


('000000033471.jpg',
 '<image>\nWhat are the colors of the bus in the image?',
 'The bus in the image is white and red.')

In [205]:
import torch

# # Load your model
# eval_model = get_peft_model(combined_model, lora_config).to(device)  # Adjust based on your setup
# eval_model.eval()  # Set the model to evaluation mode

# Example input data
image_path = image_paths[0]
text_input = text_inputs[0]  # Example text input
text_label = text_labels[0]  # Example text label (if needed for comparison)

# Prepare inputs
image_embedding = embeddings[image_path]

# Tokenize text input
input_encoding = tokenizer(
    text_inputs[0],
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=256  # Set this to match your model's input size
)

# Combine inputs for inference
input_ids = input_encoding['input_ids'].squeeze(0).to(device)  # Shape: [seq_len]
attention_mask = input_encoding['attention_mask'].squeeze(0).to(device)  # Shape: [seq_len]
image_embedding = image_embedding.squeeze(0).to(device)  # Shape: [embedding_dim]


In [206]:
image_embedding.shape

torch.Size([49, 768])

In [207]:
attention_mask.shape

torch.Size([256])

In [208]:

# Get token embeddings from PHI2 model
token_embeddings = model.get_input_embeddings()(input_ids)



In [209]:
mlp_projection = mlp_projection.to(device)
mlp_projection

MLPProjection(
  (mlp): Sequential(
    (0): Linear(in_features=768, out_features=1024, bias=False)
    (1): GELU(approximate='none')
    (2): Linear(in_features=1024, out_features=2048, bias=False)
  )
)

In [210]:
projected_image_embeddings = mlp_projection(image_embedding)

In [211]:
image_embedding.shape,token_embeddings.shape,projected_image_embeddings.shape

(torch.Size([49, 768]), torch.Size([256, 2048]), torch.Size([49, 2048]))

In [212]:
torch.cat([projected_image_embeddings, token_embeddings], dim=0).shape

torch.Size([305, 2048])

In [213]:
# # Load your model
eval_model = get_peft_model(combined_model, lora_config).to(device)  # Adjust based on your setup
eval_model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PHI2WithMLP(
      (phi2_model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 2048)
          (layers): ModuleList(
            (0-15): 16 x LlamaDecoderLayer(
              (self_attn): LlamaSdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=2048, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_

In [214]:
# Perform inference
with torch.no_grad():  # Disable gradient calculation
    outputs = eval_model(input_ids=input_ids.unsqueeze(0),  # Add batch dimension
                     attention_mask=attention_mask.unsqueeze(0),  # Add batch dimension
                     image_embeddings=None)  # Add batch dimension

# Extract predictions (modify based on your model's output)
predictions = outputs.logits  # Or the appropriate output field

# Process predictions as needed (e.g., applying softmax, argmax)
predicted_labels = torch.argmax(predictions, dim=-1)
# print(f"Predicted labels: {predicted_labels}")
# Convert predicted token IDs back to text using the tokenizer
predicted_text = tokenizer.decode(predicted_labels[0], skip_special_tokens=True)

print(f"Predicted text: {predicted_text}")

Predicted text: Tagstemplate>
  is the main of the image?
 the image?
 
?

?

?

?

?

?

?

?

?

?

?

?

?

?
?
?
?
?
?

?
?
?
?
?
?
?
?
?
?
?
?
>
>
>
?
?
?
?
?
?


>
>
>
>
?
?
?
>
>
>



>
>
>
>
}
>
>
>
>
>
>
>
>
>
>
>
}
}
}
>
>
>
}
}
}
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
ll>
>
>
>
lll

>
l}
l

>
>
lll



}
}
ll}






>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
</</>

>

>

>






























>
>
}
ь

ььььь>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
}
}
}
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>
>



In [215]:
# Perform inference
with torch.no_grad():  # Disable gradient calculation
    outputs = phi_lora_model(input_ids=input_ids.unsqueeze(0),  # Add batch dimension
                     attention_mask=attention_mask.unsqueeze(0),  # Add batch dimension
                     image_embeddings=image_embedding.unsqueeze(0))  # Add batch dimension

# Extract predictions (modify based on your model's output)
predictions = outputs.logits  # Or the appropriate output field

# Process predictions as needed (e.g., applying softmax, argmax)
predicted_labels = torch.argmax(predictions, dim=-1)
print(f"Predicted labels: {predicted_labels}")

Predicted labels: tensor([[128009, 128009,     13,    304, 128009, 128009, 128009, 128009, 128009,
         128009,    374,    264, 128009, 128009, 128009, 128009,    304,    279,
         128009, 128009, 128009, 128009, 128009,    304,    264, 128009, 128009,
         128009,    374, 128009,    304,    304, 128009, 128009,     13, 128009,
            304,    304, 128009,     13, 128009, 128009, 128009,    279, 128009,
         128009, 128009, 128009, 128009,  14924, 128009, 128009,    791,    374,
            279,   2035,    527,    279,  12235, 128009,    279,   1176, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009, 128009,
         1

In [ ]:
# outputs

In [ ]:
predictions.shape

In [216]:
# Process predictions as needed (e.g., applying softmax, argmax)
predicted_token_ids = torch.argmax(predictions, dim=-1)

# Convert predicted token IDs back to text using the tokenizer
predicted_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)

print(f"Predicted text: {predicted_text}")

Predicted text: . in is a in the in a is in in. in in. theQuestionThe is the place are the plate the first


In [ ]:
predicted_text

In [ ]:
outputs['logits'].shape

In [ ]:
if random.random() < 0.5:
    combined_embeddings = torch.cat([projected_image_embeddings, token_embeddings], dim=0)
else:
    combined_embeddings = torch.cat([token_embeddings, projected_image_embeddings], dim=0)


In [ ]:
combined_embeddings.shape

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load your model
eval_model = get_peft_model(combined_model, lora_config).to(device)  # Adjust based on your setup
eval_model.eval()  # Set the model to evaluation mode

# Example input data
image_paths = ["/kaggle/input/sample/000000000009.jpg"]
text_inputs = ["What is the capital of France?"]  # Example text input
text_labels = ["Paris"]  # Example text label (if needed for comparison)

# Prepare inputs
images = []
for path in image_paths:
    image = Image.open(path)
    inputs = clip_processor(images=image, return_tensors="pt")
    image_forward_outs = clip_model.vision_model(**inputs, output_hidden_states=True)
    image_features = feature_select(image_forward_outs)
    images.append(image_features)

# Tokenize text input
input_encoding = tokenizer(
    text_inputs[0],
    return_tensors='pt',
    padding='max_length',
    truncation=True,
    max_length=2048  # Set this to match your model's input size
)

# Combine inputs for inference
input_ids = input_encoding['input_ids'].squeeze(0).to(device)  # Shape: [seq_len]
attention_mask = input_encoding['attention_mask'].squeeze(0).to(device)  # Shape: [seq_len]
image_embedding = images[0].squeeze(0).to(device)  # Shape: [embedding_dim]

# Ensure image_embedding has the right shape for the model
# You may need to reshape or adjust the tensor based on your model's expected input
# image_embedding = image_embedding.view(1, -1)  # Adjust this if needed

# Perform inference
with torch.no_grad():  # Disable gradient calculation
    outputs = eval_model(input_ids=input_ids.unsqueeze(0),  # Add batch dimension
                     attention_mask=attention_mask.unsqueeze(0),  # Add batch dimension
                     image_embeddings=image_embedding.unsqueeze(0))  # Add batch dimension

# Extract predictions (modify based on your model's output)
predictions = outputs.logits  # Or the appropriate output field

# Process predictions as needed (e.g., applying softmax, argmax)
predicted_labels = torch.argmax(predictions, dim=-1)
print(f"Predicted labels: {predicted_labels}")


In [ ]:
!pip install trl

In [ ]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=phi_lora_model,
    train_dataset=dataset,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator
)


In [ ]:
trainer.train()

In [ ]:
# Perform inference
with torch.no_grad():  # Disable gradient calculation
    outputs = phi_lora_model(input_ids=input_ids.unsqueeze(0),  # Add batch dimension
                     attention_mask=attention_mask.unsqueeze(0),  # Add batch dimension
                     image_embeddings=image_embedding.unsqueeze(0))  # Add batch dimension

# Extract predictions (modify based on your model's output)
predictions = outputs.logits  # Or the appropriate output field

# Process predictions as needed (e.g., applying softmax, argmax)
predicted_labels = torch.argmax(predictions, dim=-1)
print(f"Predicted labels: {predicted_labels}")

In [ ]:
# Process predictions as needed (e.g., applying softmax, argmax)
predicted_token_ids = torch.argmax(predictions, dim=-1)

# Convert predicted token IDs back to text using the tokenizer
predicted_text = tokenizer.decode(predicted_token_ids[0], skip_special_tokens=True)

print(f"Predicted text: {predicted_text}")